In [1]:
import pandas as pd
import dspy
from typing import Literal

---

In [2]:
lm = dspy.LM(
    "ollama_chat/gemma3:4b", 
    api_base="http://localhost:11434", 
    api_key=""
)

dspy.configure(lm=lm)

In [3]:
class Classify(dspy.Signature):
    review: str = dspy.InputField()
    sentiment: Literal["positive", "neutral", "negative"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

In [4]:
classify = dspy.Predict(Classify)

---

In [5]:
DATASET = "../data/IMDB-Dataset.csv"

In [6]:
df = pd.read_csv(DATASET)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
positive = df[df["sentiment"] == "positive"][:30]
negative = df[df["sentiment"] == "negative"][:30]

df_sample = pd.concat([positive, negative])

---

In [13]:
%%time

df_sample[["dspy_sentiment", "dspy_confidence"]] = (
    df_sample["review"]
        .apply(lambda r: classify(review = r))
        .apply(lambda p: (p.sentiment, p.confidence))
        .apply(pd.Series)
)

CPU times: user 1.11 s, sys: 50.5 ms, total: 1.16 s
Wall time: 3.4 s


In [14]:
df_sample[df_sample["sentiment"] != df_sample["dspy_sentiment"]]

,review,sentiment,dspy_sentiment,dspy_confidence
16,Some films just simply should not be remade. T...,positive,negative,0.85
25,The Karen Carpenter Story shows a little more ...,positive,negative,0.85
29,'War movie' is a Hollywood genre that has been...,positive,neutral,0.65
30,"Taut and organically gripping, Edward Dmytryk'...",positive,negative,0.85
44,"This movie struck home for me. Being 29, I rem...",positive,negative,0.85
51,"***SPOILERS*** All too, in real life as well a...",positive,negative,0.95
11,I saw this movie when I was about 12 when it c...,negative,positive,0.85


In [15]:
df_sample.loc[51]["review"]

'***SPOILERS*** All too, in real life as well as in the movies, familiar story that happens to many young men who are put in a war zone with a gun, or rifle, in their hands. The case of young and innocent, in never handling or firing a gun, Jimmy Davis, Franchot Tone, has been repeated thousands of times over the centuries when men, like Jimmy Davis, are forced to take up arms for their country.<br /><br />Jimmy who at first wanted to be kicked out of the US Army but was encouraged to stay, by being belted in the mouth, by his good friend Fred P. Willis, Spencer Tracy, ended up on the front lines in France. With Jimmy\'s unit pinned down by a German machine gun nest he single handedly put it out of commission picking off some half dozen German soldiers from the safety of a nearby church steeple. It was when Jimmy gunned down the last surviving German, who raised his arms in surrender, that an artillery shell hit the steeple seriously wounding him.<br /><br />Recovering from his wounds 